<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/temporal_ai_pipeline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# temporal_ai_pipeline.py

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from tqdm import tqdm

# --------------------------
# 1. Synthetic Dataset
# --------------------------

class TemporalLoopDataset(Dataset):
    def __init__(self, N):
        self.X = np.column_stack([
            np.random.rand(N,1),                 # Energy E ∈ (0,1)
            np.random.uniform(-1,1,(N,1)),       # Metric M1 ∈ (-1,1)
            np.random.uniform(-1,1,(N,1)),       # Metric M2 ∈ (-1,1)
            np.random.rand(N,1),                 # Quantum factor Q1 ∈ (0,1)
            np.random.rand(N,1),                 # Quantum factor Q2 ∈ (0,1)
        ]).astype(np.float32)

        E, M1, M2, Q1, Q2 = self.X.T
        P1 = np.sin(np.pi * E) * M1 + 0.5 * Q1
        P2 = np.cos(np.pi * M2) * E + 0.3 * Q2
        P3 = E + M1 * Q1 - 0.2 * M2 * Q2
        self.Y = np.vstack([P1, P2, P3]).T.astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


# --------------------------
# 2. Physics Residual
# --------------------------

def physics_residual(pred):
    p1, p2, p3 = pred[:,0], pred[:,1], pred[:,2]
    return p1**2 + p2**2 - p3


# --------------------------
# 3. Model with MC-Dropout
# --------------------------

class TemporalAI(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, p_dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        return self.net(x)


# --------------------------
# 4. Training Utilities
# --------------------------

def train_one_epoch(model, loader, optimizer, lambda_phy):
    model.train()
    mse_loss = nn.MSELoss()
    total_loss = 0.0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)

        loss_sup = mse_loss(pred, y)
        loss_phy = torch.mean(physics_residual(pred)**2)
        loss = loss_sup + lambda_phy * loss_phy

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)

    return total_loss / len(loader.dataset)


def eval_mc_dropout(model, x, T=100):
    model.train()  # keep dropout active
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(x))
    preds = torch.stack(preds, dim=0)
    return preds.mean(dim=0), preds.std(dim=0)


# --------------------------
# 5. Main Execution
# --------------------------

if __name__ == "__main__":
    # Hyperparameters
    input_dim, hidden_dim, output_dim = 5, 32, 3
    lr = 1e-3
    batch_size = 128
    epochs = 200
    lambda_phy = 1.0
    N_samples = 10000

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare dataset and model
    dataset = TemporalLoopDataset(N_samples)
    loader  = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    model   = TemporalAI(input_dim, hidden_dim, output_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    history = {"loss": []}

    # Training loop
    for epoch in tqdm(range(1, epochs+1)):
        loss = train_one_epoch(model, loader, optimizer, lambda_phy)
        history["loss"].append(loss)

    # Save model
    os.makedirs("outputs", exist_ok=True)
    torch.save(model.state_dict(), "outputs/temporal_ai.pth")

    # --------------------------
    # 6. Monte Carlo Dropout Grid
    # --------------------------

    model.eval()
    E_vals = torch.linspace(0, 1, 50)
    M1_vals = torch.linspace(-1, 1, 50)
    grid = torch.cartesian_prod(
        E_vals, M1_vals,
        torch.tensor([0.0]),  # M2 fixed
        torch.tensor([0.5]),  # Q1 fixed
        torch.tensor([0.5])   # Q2 fixed
    ).float().to(device)

    mean_pred, std_pred = eval_mc_dropout(model, grid, T=100)

    # --------------------------
    # 7. Visualization & GIF
    # --------------------------

    fig, (ax_loss, ax_map) = plt.subplots(1, 2, figsize=(12, 5))
    norm = plt.Normalize(vmin=mean_pred[:,0].min().item(),
                        vmax=mean_pred[:,0].max().item())
    cmap = plt.cm.viridis

    def update(frame):
        ax_loss.clear()
        ax_map.clear()

        # Plot loss curve
        ax_loss.plot(history["loss"][:frame], color="crimson")
        ax_loss.set_title("Training Loss")
        ax_loss.set_xlim(0, epochs)
        ax_loss.set_ylim(0, max(history["loss"]))

        # Prepare data for scatter
        x_np    = grid[:,0].cpu().numpy()
        y_np    = grid[:,1].cpu().numpy()
        mean_np = mean_pred[:,0].cpu().numpy()
        std_np  = std_pred[:,0].cpu().numpy()
        alpha   = std_np / std_np.max()

        colors = cmap(norm(mean_np))
        colors[:, 3] = alpha

        sc = ax_map.scatter(x_np, y_np, color=colors, s=12)
        ax_map.set_title("Predicted P1 (mean color + std→alpha)")
        ax_map.set_xlabel("E")
        ax_map.set_ylabel("M1")

        return sc,

    anim = FuncAnimation(fig, update, frames=epochs, interval=50)
    gif_path = "outputs/temporal_training.gif"
    anim.save(gif_path, writer=PillowWriter(fps=20))
    plt.close(fig)

    print(f"Training complete. GIF saved to {gif_path}")